In [1]:
import pytorch_lightning as pl
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torch.utils.data import DataLoader, Dataset

import idx2numpy
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from matplotlib import pyplot as plt
import seaborn as sns
import numpy as np

In [2]:
train_images = idx2numpy.convert_from_file('data/raw/train-images-idx3-ubyte')
train_labels = idx2numpy.convert_from_file('data/raw/train-labels-idx1-ubyte')
test_images = idx2numpy.convert_from_file('data/raw/t10k-images-idx3-ubyte')
test_labels = idx2numpy.convert_from_file('data/raw/t10k-labels-idx1-ubyte')

In [3]:
batch_size = 256
epochs = 20
learning_rate = 0.1
is_transfer_learning = False

# Prepare Lightning Data Module
- Create Dataset and Wrap into Lightning Data Module
- Define dataset logic here

In [4]:
class FashinMnistDataSet(Dataset):
    def __init__(self, images, labels, transform=None):
        self.images = images
        self.labels = labels
        self.transform = transform
        
    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self, idx):
        images = self.images[idx, :]
        images = images / 255.
        images = np.expand_dims(images, axis=0)
        label = self.labels[idx]
        
        data = {
            'images': torch.tensor(images, dtype=torch.float), 'label': torch.tensor(label, dtype=torch.long)
        }
        return data

In [5]:
class FashionDataModule(pl.LightningDataModule):
    def __init__(
        self, train_images, train_labels,
        val_images, val_labels, batch_size=8,
    ) -> None:
        super().__init__()
        self.train_images = train_images
        self.train_labels = train_labels
        self.val_images = val_images
        self.val_labels = val_labels
        
        self.batch_size = batch_size
        
    def setup(self, stage: str=None) -> None:
        self.train_dataset = FashinMnistDataSet(self.train_images, self.train_labels)
        self.val_dataset = FashinMnistDataSet(self.val_images, self.val_labels)
    
    def train_dataloader(self):
        return DataLoader(
            self.train_dataset, batch_size=self.batch_size, shuffle=True,
            num_workers=4, pin_memory=True
        )
    
    def val_dataloader(self):
        return DataLoader(
            self.val_dataset, batch_size=self.batch_size, shuffle=False,
            num_workers=4, pin_memory=True
        )

In [6]:
datamodule = FashionDataModule(
    train_images, train_labels, 
    test_images, test_labels, batch_size=batch_size
)

In [7]:
datamodule.setup()

In [8]:
print(next(iter(
    datamodule.train_dataloader()
)))

{'images': tensor([[[[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          ...,
          [0.0000, 0.0000, 0.8196,  ..., 1.0000, 0.6980, 0.0000],
          [0.0000, 0.0000, 0.7373,  ..., 0.5961, 0.0941, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000]]],


        [[[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0039,  ..., 0.0000, 0.0000, 0.0000],
          ...,
          [0.0000, 0.0000, 0.6235,  ..., 0.5922, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.2431,  ..., 0.3216, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000]]],


        [[[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 

# Build model 
- We will use the lighting module class
- Using 2 different types of modules pretrain

In [9]:
from torch.optim.lr_scheduler import StepLR
from torchmetrics import Accuracy, F1Score

class FashionClassifierModule(pl.LightningModule):
    def __init__(self, num_class=10, lr=0.001, freeze_weights=False):
        super(FashionClassifierModule, self).__init__()
        # self.backbone = torchvision.models.resnet50(weights="IMAGENET1K_V1")
        self.backbone = torchvision.models.resnet50(pretrained=False)
        if freeze_weights:
            for param in self.backbone.parameters():
                param.requires_grad = False
        
        self.backbone.conv1 = nn.Conv2d(1, 64, 3, 1)
        self.backbone.fc = nn.Linear(self.backbone.fc.in_features, num_class)
        
        self.lr = lr
        self.criterion = nn.CrossEntropyLoss()
        
        # Define metrics
        self.train_acc = Accuracy(task='multiclass', num_classes=num_class)
        self.val_acc = Accuracy(task='multiclass', num_classes=num_class)
        
        self.train_f1 = F1Score(task='multiclass', num_classes=num_class)
        self.val_f1 = F1Score(task='multiclass', num_classes=num_class)
        
    def forward(self, x):
        return self.backbone(x)
    
    def training_step(self, batch, batch_idx):
        logits = self(batch['images'])
        loss = self.criterion(logits, batch['label'])
        
        self.train_acc(batch['label'], logits.argmax(dim=1))
        self.train_f1(batch['label'], logits.argmax(dim=1))
        
        self.log_dict({
            'train_acc': self.train_acc,
            'train_f1': self.train_f1
        }, prog_bar=True, logger=True)
        
        return loss
    
    def validation_step(self, batch, batch_idx):
        logits = self(batch['images'])
        loss = self.criterion(logits, batch['label'])
        
        self.val_acc(batch['label'], logits.argmax(dim=1))
        self.val_f1(batch['label'], logits.argmax(dim=1))
        
        self.log_dict({
            'val_loss': loss.item(),
            'val_acc': self.val_acc,
            'val_f1': self.val_f1
        }, prog_bar=True, logger=True)
    
    def test_step(self, batch, batch_idx):
        return self.validation_step(batch, batch_idx)
    
    def configure_optimizers(self):
        optim = torch.optim.SGD(self.parameters(), lr=self.lr, momentum=0.9)
        scheduler = StepLR(optim, step_size=1, gamma=0.7)
        
        return [optim], [scheduler]

In [10]:
model = FashionClassifierModule(freeze_weights=False, lr=learning_rate)

In [11]:
model_checkpoint = pl.callbacks.ModelCheckpoint(
    monitor="val_f1", save_top_k=5, dirpath='ckpt/', 
    save_last=False, filename="{epoch:02d}-{val_f1:.4f}",
    mode='max'
)
early_stopping = pl.callbacks.EarlyStopping(monitor="val_f1", mode='max', patience=5)

trainer = pl.Trainer(
    accelerator='auto', devices=[0],
    callbacks=[model_checkpoint, early_stopping], max_epochs=epochs
)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


HPU available: False, using: 0 HPUs


In [12]:
trainer.fit(model, datamodule)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type               | Params
-------------------------------------------------
0 | backbone  | ResNet             | 23.5 M
1 | criterion | CrossEntropyLoss   | 0     
2 | train_acc | MulticlassAccuracy | 0     
3 | val_acc   | MulticlassAccuracy | 0     
4 | train_f1  | MulticlassF1Score  | 0     
5 | val_f1    | MulticlassF1Score  | 0     
-------------------------------------------------
23.5 M    Trainable params
0         Non-trainable params
23.5 M    Total params
94.079    Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=20` reached.


In [13]:
trainer.test(dataloaders=datamodule.val_dataloader())

/home/superceed1/anaconda3/envs/py39torch110/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/checkpoint_connector.py:124: UserWarning: `.test(ckpt_path=None)` was called without a model. The best model of the previous `fit` call will be used. You can pass `.test(ckpt_path='best')` to use the best model or `.test(ckpt_path='last')` to use the last model. If you pass a value, this warning will be silenced.
  rank_zero_warn(
Restoring states from the checkpoint path at /mnt/882eb9b0-1111-4b8f-bfc3-bb89bc24c050/pytorch/mnist_onnx/ckpt/epoch=18-val_f1=0.8830.ckpt
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Loaded model weights from checkpoint at /mnt/882eb9b0-1111-4b8f-bfc3-bb89bc24c050/pytorch/mnist_onnx/ckpt/epoch=18-val_f1=0.8830.ckpt


Testing: 0it [00:00, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│          val_acc          │    0.8830000162124634     │
│          val_f1           │    0.8830000162124634     │
│         val_loss          │    0.3379245102405548     │
└───────────────────────────┴───────────────────────────┘

[{'val_loss': 0.3379245102405548,
  'val_acc': 0.8830000162124634,
  'val_f1': 0.8830000162124634}]